# Exploring how to extract various metadata from .tif files using `tifffile`

In [1]:
import tifffile


# Leica Matrix Screener `.ome.tif` file acquired on SP5

In [2]:
mscreener = tifffile.TiffFile("C:/Users/Volker/Data/AndreAlves_Subset/chamber--U02--V05/field--X00--Y01/image--L0000--S00--U02--V05--J08--E00--O00--X00--Y01--T0000--Z01--C00.ome.tif")

In [4]:
mscreener.pages.pages[0].is_lsm

False

In [5]:
mscreener.pages.pages[0].is_ome

True

In [6]:
mscreener.ome_metadata

{'Experiment': {'Description': 'Experiment',
  'ExperimenterRef': {'ID': 'Experimenter:0'},
  'Type': 'Other',
  'ID': 'Experiment:0'},
 'Experimenter': {'FirstName': 'Frank',
  'LastName': 'Sieckmann',
  'Email': 'Frank.Sieckmann@leica-microsystems.com',
  'Institution': 'Leica Microsystems CMS GmbH',
  'OMEName': 'Frank',
  'GroupRef': {'ID': 'Group:0'},
  'ID': 'Experimenter:0'},
 'Group': {'Leader': {'ID': 'Experimenter:0'},
  'Contact': {'ID': 'Experimenter:0'},
  'ID': 'Group:0',
  'Name': 'MatrixScreener Group'},
 'Instrument': {'Microscope': {'Manufacturer': 'Leica Microsystems',
   'Model': 'DMI6000 Turret',
   'SerialNumber': 5100000872,
   'Type': 'Unknown'},
  'LightSource': {'Laser': {'Wavelength': 405,
    'LaserMedium': 'Unknown',
    'Type': 'Unknown'},
   'ID': 'LightSource:0',
   'Manufacturer': 'Leica Microsystems',
   'Model': 'Unknown',
   'SerialNumber': '',
   'Power': 53.0000610388818},
  'Detector': [{'ID': 'Detector:1',
    'Manufacturer': 'Leica Microsystems'

# 3i Lattice Lightsheet operated and capturing with Janelia Labview software

In [7]:
lls = tifffile.TiffFile("C:/Users/Volker/Dropbox/Github/Lattice_Lightsheet_Deskew_Deconv/testset/drp1_dendra2_test_1_CamA_ch0_stack0000_488nm_0000000msec_0018218290msecAbs.tif")

c:\users\volker\anaconda3\envs\spimenv\lib\site-packages\tifffile\tifffile.py:4267: UserWarning: tag 32781: coercing invalid ASCII to bytes
  warnings.warn('tag %i: coercing invalid ASCII to bytes' % code)


Note that we already got a warning about tiff tag 32781 being malformed ...

In [10]:
lls.pages.pages

In [13]:
tags = lls.pages.pages[0].tags

In [14]:
for t in tags:
    print(f"{t}: {tags[t]}")

ImageWidth: TiffTag 256 ImageWidth  1H @278928  256
ImageLength: TiffTag 257 ImageLength  1H @278928  512
BitsPerSample: TiffTag 258 BitsPerSample  1H @278928  16
Compression: TiffTag 259 Compression  1H @278928  NONE
PhotometricInterpretation: TiffTag 262 PhotometricInterpretation  1H @278928  MINISBLACK
FillOrder: TiffTag 266 FillOrder  1H @278928  MSB2LSB
StripOffsets: TiffTag 273 StripOffsets  1I @278928  (16558,)
Orientation: TiffTag 274 Orientation  1H @278928  TOPLEFT
SamplesPerPixel: TiffTag 277 SamplesPerPixel  1H @278928  1
RowsPerStrip: TiffTag 278 RowsPerStrip  1H @278928  512
StripByteCounts: TiffTag 279 StripByteCounts  1I @278928  (262144,)
XResolution: TiffTag 282 XResolution  2I @278924  (1073741824, 1073741824)
YResolution: TiffTag 283 YResolution  2I @278932  (1073741824, 1073741824)
PlanarConfiguration: TiffTag 284 PlanarConfiguration  1H @278948  CONTIG
ResolutionUnit: TiffTag 296 ResolutionUnit  1H @278948  NONE
Software: TiffTag 305 Software  29s @278940  Nationa

In [16]:
lls.pages.pages[0].imagelength

512

In [17]:
lls.pages.pages[0].imagewidth

256

In [18]:
lls.pages.pages[0].imagedepth

1

In [21]:
lls.asarray().shape

(76, 512, 256)